In [1]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")

/root/autodl-tmp/conda/envs/bert/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the module from /root/.cache/huggingface/modules/datasets_modules/datasets/yelp_review_full/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf (last modified on Thu Jun 15 14:53:25 2023) since it couldn't be found locally at yelp_review_full., or remotely on the Hugging Face Hub.
Found cached dataset yelp_review_full (/root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00, 31.15it/s]


In [4]:
from transformers import AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-9f3f59e35328fe67.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-2f29d9b66e1106d7.arrow


In [7]:
tokenized_datasets["train"]

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 650000
})

In [8]:
tokenized_datasets["test"]

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50000
})

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-3d6fd8b5509303c4.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-6f9a8310baccff89.arrow


In [10]:
from transformers import TrainingArguments

In [11]:
training_args = TrainingArguments(output_dir="test_trainer")

In [12]:
import numpy as np

In [13]:
import evaluate

In [14]:
metric = evaluate.load("accuracy")

Using the latest cached version of the module from /root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Thu Jun 15 18:51:36 2023) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

基于transformer的神经网络方法：

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [17]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=10)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

/root/autodl-tmp/conda/envs/bert/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/root/autodl-tmp/conda/envs/bert/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.987765,0.563550
2,0.963300,0.911858,0.602000
3,0.963300,1.119259,0.599500
4,0.564700,1.294533,0.594700
5,0.265800,1.554174,0.591950
6,0.265800,1.823240,0.605750
7,0.131700,2.107365,0.598950
8,0.060500,2.393476,0.603450
9,0.060500,2.584165,0.601500
10,0.022200,2.614256,0.604250


/root/autodl-tmp/conda/envs/bert/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/root/autodl-tmp/conda/envs/bert/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/root/autodl-tmp/conda/envs/bert/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/root/autodl-tmp/conda/envs/bert/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was a

TrainOutput(global_step=3130, training_loss=0.32154104130717515, metrics={'train_runtime': 2628.2837, 'train_samples_per_second': 38.048, 'train_steps_per_second': 1.191, 'total_flos': 2.63118142464e+16, 'train_loss': 0.32154104130717515, 'epoch': 10.0})

In [30]:
import torch
torch.save(model, 'fine_tune_10_epoch_from_bert_base.pth')

传统方法：（决策树）

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from datasets import Dataset

# 获取训练集和测试集的特征和标签
train_features = small_train_dataset['text']
train_labels = small_train_dataset['label']
test_features = small_eval_dataset['text']
test_labels = small_eval_dataset['label']

# 特征提取
vectorizer = CountVectorizer(max_features=1000)
train_features_vectorized = vectorizer.fit_transform(train_features)
test_features_vectorized = vectorizer.transform(test_features)

# 训练决策树模型
clf = DecisionTreeClassifier()
clf.fit(train_features_vectorized, train_labels)

# 在测试集上进行预测并评估模型性能
pred_labels = clf.predict(test_features_vectorized)
accuracy = accuracy_score(test_labels, pred_labels)
report = classification_report(test_labels, pred_labels)
print('Accuracy:', accuracy)
print('Classification report:\n', report)

Accuracy: 0.3381
Classification report:
               precision    recall  f1-score   support

           0       0.45      0.44      0.45      4049
           1       0.28      0.28      0.28      3950
           2       0.27      0.27      0.27      3983
           3       0.29      0.29      0.29      4038
           4       0.40      0.40      0.40      3980

    accuracy                           0.34     20000
   macro avg       0.34      0.34      0.34     20000
weighted avg       0.34      0.34      0.34     20000



传统方法：（贝叶斯）

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# 获取训练集和测试集的特征和标签
train_features = small_train_dataset['text']
train_labels = small_train_dataset['label']
test_features = small_eval_dataset['text']
test_labels = small_eval_dataset['label']

# 特征提取
vectorizer = CountVectorizer(max_features=1000)
train_features_vectorized = vectorizer.fit_transform(train_features)
test_features_vectorized = vectorizer.transform(test_features)

# 训练朴素贝叶斯模型
clf = MultinomialNB()
clf.fit(train_features_vectorized, train_labels)

# 在测试集上进行预测并评估模型性能
pred_labels = clf.predict(test_features_vectorized)
accuracy = accuracy_score(test_labels, pred_labels)
report = classification_report(test_labels, pred_labels)
print('Accuracy:', accuracy)
print('Classification report:\n', report)

Accuracy: 0.5022
Classification report:
               precision    recall  f1-score   support

           0       0.60      0.66      0.63      4049
           1       0.42      0.34      0.38      3950
           2       0.42      0.43      0.42      3983
           3       0.45      0.45      0.45      4038
           4       0.58      0.63      0.60      3980

    accuracy                           0.50     20000
   macro avg       0.50      0.50      0.50     20000
weighted avg       0.50      0.50      0.50     20000

